In [1]:
import sqlite3
from pprint import pprint
import pandas as pd
import math
import datetime
import matplotlib.pyplot as plt

In [2]:
tInicioGlobal = datetime.datetime.now()

## INICIO extraer datos (BBDD)

In [3]:
### Inicio conexion BBDD
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["stum_for_you"]
mycol = mydb["multidimensional"]
mycol.find_one()

{'_id': ObjectId('5e2040ad37d0e4054c2cf0fb'),
 'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1},
 'fecha': {'anyo': '2019', 'mes': '01', 'dia': '23'},
 'juego': {'puntero_juego': 9477,
  'genero': ['racing'],
  'precio_juego': 10,
  'clasificacion': 'INDIE'},
 'proveedor': {'puntero_proveedor': 29,
  'pais': 'New Zealand',
  'valoracion': 1}}

In [4]:
# cursor.execute("SELECT DISTINCT s.problem_id FROM Submission as s GROUP BY s.user_id, s.problem_id HAVING (count(s.status='AC')+count(s.status='PE'))>0")
# tablaProblemBBDD = cursor.fetchall()
# cursor.execute("SELECT DISTINCT s.user_id FROM Submission as s GROUP BY s.user_id, s.problem_id HAVING (count(s.status='AC')+count(s.status='PE'))>0")
# tablaUsersBBDD = cursor.fetchall()
dataMultidimensional = mycol.find()
tablaUsersBBDD = []
tablaProblemBBDD = []
dataParse = []
for x in dataMultidimensional:
    if 'proveedor' in x and 'juego' in x and 'movimiento' in x:
        proveedor = x['proveedor']
        juego = x['juego']
        movimiento = x['movimiento']
        if 'puntero_proveedor' in proveedor and 'puntero_juego' in juego and 'tipo_movimiento' in movimiento:
            if movimiento['tipo_movimiento'] == 'compra':
                dataParse.append({
                    'puntero_juego': juego['puntero_juego'],
                    'puntero_proveedor': proveedor['puntero_proveedor'],
                    'cantidad': movimiento['cantidad'],
                    'fecha': x['fecha']['anyo'] + '-' + x['fecha']['mes'] + '-' + x['fecha']['dia']
                })
                tablaUsersBBDD.append(proveedor['puntero_proveedor'])
                tablaProblemBBDD.append(juego['puntero_juego'])
tablaUsersBBDD = list(set(tablaUsersBBDD))
tablaProblemBBDD = list(set(tablaProblemBBDD))

## FIN extraer datos

## INICIO parsear datos

In [6]:
dfSubmissionParaEntrenar = dfSubmission.copy()

In [7]:
dfUsersProblem = pd.DataFrame(0, index=tablaUsersBBDD, columns=tablaProblemBBDD)

for i in dfSubmissionParaEntrenar.itertuples():
    dfUsersProblem.at[i.puntero_proveedor, i.puntero_juego] += int(i.cantidad)

In [8]:
dfOriginalUsersProblem = dfUsersProblem.copy()
aux = dfUsersProblem.sum()
aux = aux[aux<1]
dfUsersProblem = dfUsersProblem.drop(aux.index ,axis=1)
aux = dfUsersProblem.transpose().sum()
aux = aux[aux<1]
dfUsersProblem = dfUsersProblem.drop(aux.index ,axis=0)

In [9]:
dfOriginalUsersProblemInv = dfOriginalUsersProblem.copy()
dfOriginalUsersProblemInv[dfOriginalUsersProblem==0] = 1
dfOriginalUsersProblemInv[dfOriginalUsersProblem==1] = 0
dfUsersProblemInv = dfUsersProblem.copy()
dfUsersProblemInv[dfUsersProblem==0] = 1
dfUsersProblemInv[dfUsersProblem==1] = 0

## FIN parsear datos

## INICIO sacar lista de recomendación

#### Calculo probabilidad simple

In [10]:
def calculoProbabilidadSimple():
    return dfUsersProblem.sum()/len(dfUsersProblemInv.index)
sProbSimple = calculoProbabilidadSimple()

#### probabilidad de resolver el problema COLUMNA habiendo resuelto el problema FILA

In [11]:
def calculoProbabilidadCondicionadaRR():
    dfProblemProbabilidadSiOptimizado = pd.DataFrame(index=dfUsersProblem.columns, columns=dfUsersProblem.columns)
    for column in dfUsersProblem.columns:
        denNo = dfUsersProblem[column].sum() + 2
        dfProblemProbabilidadSiOptimizado[column] = ((dfUsersProblem.mul(dfUsersProblem[column], axis=0)).sum() + 1) / denNo
    return dfProblemProbabilidadSiOptimizado
dfProblemProbabilidadSiOptimizado = calculoProbabilidadCondicionadaRR()
# dfProblemProbabilidadSiOptimizado.to_csv("probPriori.csv")

tInicio = datetime.datetime.now()
dfProblemProbabilidadSiOptimizado = pd.read_csv("probPriori.csv", index_col=0, parse_dates=True)
tFinal = datetime.datetime.now()
t = tFinal - tInicio
print(t)

#### Calculo para TODOS los usuarios

def calculoTablaProbTodosLosUsuarios():
    dfFinal = pd.DataFrame()
    for c in dfProblemProbabilidadSiOptimizado.columns:
        aux = dfOriginalUsersProblem.mul(dfProblemProbabilidadSiOptimizado[c])
        aux = aux[aux!=0].prod(axis=1)
        aux2 = dfOriginalUsersProblemInv.mul(1-dfProblemProbabilidadSiOptimizado[c])
        aux2 = aux2[aux2!=0].prod(axis=1)
        dfFinal[c]=aux*aux2*sProbSimple[c]
    dfFinal.to_csv("allUsersProb.csv")
    return dfFinal
dfFinal = calculoTablaProbTodosLosUsuarios()

In [ ]:
#### Cargar de archivo los datos generados por la funcion anterior (para la demo porque la funcion tarda mucho)
dfFinal = pd.read_csv("allUsersProb.csv", index_col=0, parse_dates=True)
dfFinal.columns = dfFinal.columns.astype(int)

In [19]:
#Ordenar recomendaciones para cada usuario
def generarListaDeRecomendacionOrdenada():
    listaRecomendacion = pd.DataFrame(columns=[0],index=dfFinal.index)
    for u in dfFinal.index:
        listaRecomendacion.at[u,0] = dfFinal.loc[u][dfOriginalUsersProblem.loc[u]!=1].copy().sort_values(ascending=False)
    return listaRecomendacion
listaRecomendacion = generarListaDeRecomendacionOrdenada()

1     403       6.221661e-08
6955      2.796995e-48
...
2     11629     5.095293e-06
3027      4.231885e-28
...
3     11344     1.088363e-07
6870      1.189373e-13
...
4     3575     2.651497e-196
7023     3.781733e-211
...
5     7023     8.024027e-142
200      1.192465e-146
...
6     7164      7.756619e-09
12206     2.525474e-09
...
7     3575     5.126638e-173
3860     3.233967e-176
...
8     3575     6.815033e-186
3860     4.630716e-187
...
9     7023     3.721861e-170
200      3.076322e-174
...
10    3193      1.094759e-01
1498      1.364878e-03
...
11    3575     1.672064e-182
3860     7.037581e-186
...
12    3575     5.720840e-177
7023     2.471593e-179
...
13    3575     5.634984e-188
3860     4.383778e-193
...
14    9355      1.817314e+16
51        7.613503e+15
...
15    11490     1.616259e+15
6181      6.427726e+06
...
16    3575     2.738468e-178
7023     1.217810e-178
...
17    3296      3.091913e-01
7316      5.470825e-05
...
18    3575     1.158442e-170
7023     7.929244e-

In [20]:
siguienteJuegoDeCadaProveedor = {}
for x in listaRecomendacion.index:
    siguienteJuegoDeCadaProveedor[x] = listaRecomendacion.at[x,0].index[0]

siguienteJuegoDeCadaProveedor

{1: 403,
 2: 11629,
 3: 11344,
 4: 3575,
 5: 7023,
 6: 7164,
 7: 3575,
 8: 3575,
 9: 7023,
 10: 3193,
 11: 3575,
 12: 3575,
 13: 3575,
 14: 9355,
 15: 11490,
 16: 3575,
 17: 3296,
 18: 3575,
 19: 3575,
 20: 115,
 21: 3575,
 22: 3575,
 23: 200,
 24: 8595,
 25: 9772,
 26: 200,
 27: 1334,
 28: 3031,
 29: 5299,
 30: 3575,
 31: 12126,
 32: 3575,
 33: 12439,
 34: 3575,
 35: 3575,
 36: 4367,
 37: 3860,
 38: 7023,
 39: 3575,
 40: 3575}

In [14]:
def recomendacionUnUsuario(user):
    return listaRecomendacion.at[user,0]

In [15]:
tFinalGlobal = datetime.datetime.now()
t = tFinalGlobal - tInicioGlobal
print(t)

2:23:25.631157


## FIN sacar lista de recomendación